In [ ]:
pip install lxml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas  as pd

auto_df= pd.read_csv('/content/drive/MyDrive/MyDrive/auto_df.csv', encoding='utf-8')

In [ ]:
auto_df

,Unnamed: 0,week,prodname,온도,습도,order_quant,sold_quant,건설기성액,국내건설수주액
0,0,2021-04-26,AE,NaN,NaN,1.190733e+04,NaN,NaN,NaN
1,1,2021-04-26,PEMA-SPR,NaN,NaN,-9.658918e-09,NaN,NaN,NaN
2,2,2021-04-26,PEMA-HR1000,NaN,NaN,2.994239e+05,NaN,NaN,NaN
3,3,2021-04-26,SRE-200,NaN,NaN,8.212241e-18,NaN,NaN,NaN
4,4,2021-04-26,PEMA-HR1500,NaN,NaN,1.206527e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
373,373,2021-07-26,CSA5000,NaN,NaN,2.124372e+05,NaN,NaN,NaN
374,374,2021-07-26,PEMA-500FR,NaN,NaN,4.667127e+04,NaN,NaN,NaN
375,375,2021-07-26,PEMA-PCM2000,NaN,NaN,1.889885e+04,NaN,NaN,NaN
376,376,2021-07-26,PEMA-SR3000F,NaN,NaN,5.919998e+04,NaN,NaN,NaN


In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') # 한글폰트

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import json
import requests
import xml.etree.ElementTree as ET
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import pprint
from pandas import DataFrame
import datetime
from datetime import datetime, timedelta

In [ ]:
auto_df

,Unnamed: 0,week,prodname,온도,습도,order_quant,sold_quant,건설기성액,국내건설수주액
0,0,2021-04-26,AE,NaN,NaN,1.190733e+04,NaN,NaN,NaN
1,1,2021-04-26,PEMA-SPR,NaN,NaN,-9.658918e-09,NaN,NaN,NaN
2,2,2021-04-26,PEMA-HR1000,NaN,NaN,2.994239e+05,NaN,NaN,NaN
3,3,2021-04-26,SRE-200,NaN,NaN,8.212241e-18,NaN,NaN,NaN
4,4,2021-04-26,PEMA-HR1500,NaN,NaN,1.206527e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
373,373,2021-07-26,CSA5000,NaN,NaN,2.124372e+05,NaN,NaN,NaN
374,374,2021-07-26,PEMA-500FR,NaN,NaN,4.667127e+04,NaN,NaN,NaN
375,375,2021-07-26,PEMA-PCM2000,NaN,NaN,1.889885e+04,NaN,NaN,NaN
376,376,2021-07-26,PEMA-SR3000F,NaN,NaN,5.919998e+04,NaN,NaN,NaN


In [ ]:
current_time = datetime.now()
# 현재시점을 불러옴
formatted__date = current_time.strftime("%Y%m")
# 건설api 형식으로 수정함 (년/월)
yesterday = current_time - timedelta(days=1)
# 어제시점을 불러옵니다 (기상청은 현재시점에 전날까지 업데이트)
w_date = yesterday.strftime("%Y%m%d")
# 기상청 api 형식으로 수정함(년/월/일)

In [ ]:
print(formatted__date)
print(w_date)

202308
20230813


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import json
import requests
import xml.etree.ElementTree as ET
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import pprint
from pandas import DataFrame
import datetime
from datetime import datetime, timedelta


current_time = datetime.now()
# 현재시점을 불러옴
formatted__date = current_time.strftime("%Y%m")
# 건설api 형식으로 수정함 (년/월)
yesterday = current_time - timedelta(days=1)
# 어제시점을 불러옵니다 (기상청은 현재시점에 전날까지 업데이트)
w_date = yesterday.strftime("%Y%m%d")
# 기상청 api 형식으로 수정함(년/월/일)

data_dict = {
    '901Y104' : '건설기성액',
    '901Y020' : '국내건설수주액'
}

KEY = 'P6B82DH7S3NJPJF38DPV'
PERIOD = 'M'
START_DATE = '202104'
END_DATE = formatted__date

def get_product(KEY, STAT_CD, PERIOD, START_DATE, END_DATE):
    url = 'http://ecos.bok.or.kr/api/StatisticSearch/{}/xml/kr/1/1000/{}/{}/{}/{}/'.format(KEY, STAT_CD, PERIOD, START_DATE, END_DATE)

    response = requests.get(url).content.decode('utf-8')
    xml_obj = BeautifulSoup(response, 'lxml-xml')
    rows = xml_obj.findAll("row")
    return rows


item_list = [
    'STAT_CODE','STAT_NAME','ITEM_CODE1','ITEM_NAME1','ITEM_CODE2','ITEM_NAME2','ITEM_CODE3','ITEM_NAME3','UNIT_NAME',
    'TIME','DATA_VALUE'
]

result_list = list()

for k in data_dict.keys():
    rows = get_product(KEY , k ,PERIOD , START_DATE , END_DATE)

    for p in range(0, len(rows)):
        info_list = list()

        for i in item_list:
            try:
                individual_info = rows[p].find(i).text
            except:
                individual_info = ""
            info_list.append(individual_info)

        result_list.append(info_list)

result_df = pd.DataFrame(result_list , columns = ['통계표코드', '통계명', '통계 항목 1코드', '통계항목2코드', '통계항목명2', '통계항목3코드', '뭔데그럼', '나머지두개는', '단위', '시점', '값']).drop_duplicates()


gunsal_gisung_df = result_df[(result_df['통계명'] == '8.4.2. 건설기성액') &
                            (result_df['통계항목2코드'] == '총기성액') &
                            (result_df['통계항목3코드'] == '경상')].copy()

gunsal_suzu_df = result_df[(result_df['통계명'] == '8.4.1. 국내건설수주액') &
                          (result_df['통계항목2코드'] == '총수주액')].copy()

def extract_year_month(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.year, date_obj.month

# autodf의 년월 추출
auto_df['year'], auto_df['month'] = zip(*auto_df['week'].map(extract_year_month))

# 년월이 일치하는 경우 값을 할당
for _, row in auto_df.iterrows():
    matching_row = gunsal_suzu_df[gunsal_suzu_df['시점'] == f"{row['year']}{row['month']:02d}"]
    if not matching_row.empty:
        auto_df.loc[_, '국내건설수주액'] = matching_row['값'].values[0]

for _, row in auto_df.iterrows():
    matching_row = gunsal_gisung_df[gunsal_gisung_df['시점'] == f"{row['year']}{row['month']:02d}"]
    if not matching_row.empty:
        auto_df.loc[_, '건설기성액'] = matching_row['값'].values[0]




url2 = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params = {
    'serviceKey': 'm6B9bWsl85oDiUgVP3Fkfrgdj89/ea59MvlX6XqTmaurYr2EhziN/Sii8RhhNULFgxQWj/DudnIBBN4rr1iWeg==',
    'pageNo': '1~80',
    'numOfRows': '900',
    'dataType': 'XML',
    'dataCd': 'ASOS',
    'dateCd': 'DAY',
    'startDt': '20210401',
    'endDt': w_date,   # 현재날짜에 하루전까지만 업데이트 됨
    'stnIds': '232' #천안232
}


w_response = requests.get(url2, params=params)



# 응답 데이터를 XML로 파싱
root = ET.fromstring(w_response.content)

# 데이터를 저장할 리스트 생성
data = []

# XML에서 데이터 추출하여 리스트에 추가
for item in root.iter('item'):
    TA_AVG = item.find('avgTa').text
    avgRhm = item.find('avgRhm').text
    tm = item.find('tm').text


    # 데이터를 딕셔너리 형태로 저장
    data.append({
        'tm': tm,
        'TA_AVG': TA_AVG,
        'avgRhm': avgRhm,

    })

# 데이터프레임 생성
df = pd.DataFrame(data)

for _, row in df.iterrows():
    matching_row = auto_df[auto_df['week']== row['tm']]
    if not matching_row.empty:
        auto_df.loc[matching_row.index,'온도'] = row['TA_AVG']
        auto_df.loc[matching_row.index,'습도'] = row['avgRhm']



auto_df














,Unnamed: 0,week,prodname,온도,습도,order_quant,sold_quant,건설기성액,국내건설수주액,year,month
0,0,2021-04-26,AE,15.1,47.5,1.190733e+04,NaN,11514860,17774868,2021,4
1,1,2021-04-26,PEMA-SPR,15.1,47.5,-9.658918e-09,NaN,11514860,17774868,2021,4
2,2,2021-04-26,PEMA-HR1000,15.1,47.5,2.994239e+05,NaN,11514860,17774868,2021,4
3,3,2021-04-26,SRE-200,15.1,47.5,8.212241e-18,NaN,11514860,17774868,2021,4
4,4,2021-04-26,PEMA-HR1500,15.1,47.5,1.206527e+05,NaN,11514860,17774868,2021,4
...,...,...,...,...,...,...,...,...,...,...,...
373,373,2021-07-26,CSA5000,28.1,68.8,2.124372e+05,NaN,10992998,14541827,2021,7
374,374,2021-07-26,PEMA-500FR,28.1,68.8,4.667127e+04,NaN,10992998,14541827,2021,7
375,375,2021-07-26,PEMA-PCM2000,28.1,68.8,1.889885e+04,NaN,10992998,14541827,2021,7
376,376,2021-07-26,PEMA-SR3000F,28.1,68.8,5.919998e+04,NaN,10992998,14541827,2021,7


In [ ]:
# CSV 파일로 저장
auto_df.to_excel('api_자동화.xlsx', index=False)
